Created on Monday 04 January 2021  

# **Crawler interne au site - POC filtre par date**
**Group 2 - Recherche de nouvelles sources**  
*Projet Inter-Promo 2021 de la formation SID, Université Paul Sabatier, Toulouse*

@authors : Sibel Yüksel, Rémy Lapeyre, Maël Lesavourey, Nicolas Enjalbert Courrech

Lorsqu'une source a été définie comme pertinente, il est important de l'exploiter en recherchant les articles qui la composent. Pour cela, une recherche par site est effectuée pour récupérer les derniers articles répondant à une requête, dans notre cas à une paire de mots clefs répondant au thème de recherche (innovation et gamme de gestion). Cette récolte est prévue d'être faite tous les jours à heure fixe afin d'être au plus proche des informations. 

La solution basique mise en place est de faire un robot spécifique à chaque site web qui va retourner les articles du site répondant à une requête textuelle quand le site permet de mettre en place ce type de recherche. Or une partie de ces sites ne permet pas de faire une sélection dans la recherche de la date de publication d'un de leur article. Il revient donc de récupérer l'ensemble des articles du site, de les scraper (récolter l'information de l'article comme le contenu, la date) et enfin de les trier par date pour ne prendre que ceux publiés dans les dernières 24h. Or cette méthode, certes juste, n 'est pas optimale car elle demande de scraper inutilement des articles qui sont déjà théoriquement référencés en base de données. 

Pour optimiser cette chaîne de traitement, le groupe 1 a décidé d'explorer de préselection en ne prenant que le top n des articles des sites et de faire le filtrage par date (comme décrit dans le paragraphe ci dessus). 
Nous avons choisi d'explorer la piste de filtrage par date en amont du scraping. Cette piste est assez complexe car elle demande d'être généraliste sur des informations qui sont différentes d'un site à l'autre. 

Ce document montre comment nous avons essayé de mettre en place cette sélection sur un crawler spécifique par site. 


# Librairies

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from tqdm import tqdm
from datetime import datetime
from datetime import date

# QUERY INSIDE SITE

In [32]:
path: str = 'C:/Users/Y-Sib/OneDrive/Documents/Applications/chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(path)
driver.get(url_site_5)

In [35]:
def get_query_src_page(pages: int, keywords: list, source_url: str, 
                       search_url: str, url_attr: str = None, 
                       separator: str = '+') -> list:
    """Documentation

    Get the source code of the pages resulting a research of a keyword.

    Parameters:
        pages: The number of pages you want to retrieve.
        keywords: List of keywords used in the research.
        source_url: URL of the website.
        search_url: The part of URL before keywords (ex: search?q=).
        url_attr: PHP attributes that may be after the keyword.
        separator: The separator between two keywords.

    Out:
        page_list: List containing the source code of each page of the research
    """
    # Initialize webdriver
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    wd = webdriver.Chrome(path, options=chrome_options)

    # Base url
    base = source_url + search_url
    if url_attr is None:
        for kw in keywords:
            base += kw + separator
        base = base[:-len(separator)]
    else:
        for kw in keywords:
            base += kw + separator
        base = base[:-len(separator)]
        base += '/' + url_attr
    page_list = []
    """ 
    This loop may be used when you will treat page switching.
    Note that you will need a condition to distinguish 2 cases : 
        -the switch is managed by URL.
        -the switch is managed dynamically.
        
    for i in tqdm(range(pages)):
        k = i + 1
        # Simulate a webdriver instance and get the source page 
        wd.get(base + '&page=' + str(k))
        # Transform it with the BS' html parser
        soup = BeautifulSoup(wd.page_source)
        page_list.append(soup)
    """
        
    # Simulate a webdriver instance and get the source page 
    wd.get(base)
    # Transform it with the BS' html parser
    soup = BeautifulSoup(wd.page_source)
    page_list.append(soup)
    
    return page_list



In [36]:
pages = get_query_src_page(pages = 1, keywords = ['ressources', 'humaines'],
                           source_url = 'https://grh-multi.net/', 
                           search_url = '?s=')

pages

[<html lang="fr-FR" prefix="og: http://ogp.me/ns#"><head>
 <meta charset="utf-8"/>
 <meta content="width=device-width, initial-scale=1" name="viewport"/>
 <link href="http://gmpg.org/xfn/11" rel="profile"/>
 <link href="https://grh-multi.net/xmlrpc.php" rel="pingback"/>
 <title>Vous avez cherché ressources humaines - M2 GRH EMN</title>
 <!-- This site is optimized with the Yoast SEO plugin v4.8 - https://yoast.com/wordpress/plugins/seo/ -->
 <meta content="noindex,follow" name="robots"/>
 <link href="https://grh-multi.net/fr/search/ressources+humaines/" rel="canonical"/>
 <link href="https://grh-multi.net/fr/search/ressources+humaines/page/2/" rel="next"/>
 <meta content="fr_FR" property="og:locale"/>
 <meta content="en_GB" property="og:locale:alternate"/>
 <meta content="object" property="og:type"/>
 <meta content="Vous avez cherché ressources humaines - M2 GRH EMN" property="og:title"/>
 <meta content="https://grh-multi.net/fr/search/ressources+humaines/" property="og:url"/>
 <meta c

In [38]:
def get_art_url(page_list: list, source_url: str, tag: str, 
                attr_key: str = None, attr_value: str = None) -> list:
    """Documentation:

    Get the article's urls.
   
    Parameters:
        page_list: list of pages' source code.
        source_url: URL of source website.
        tag:  tag of the closest identifiable tag.
        attr_key: attribute key of the closest identifiable tag.
        attr_value: attribute value of the closest identifiable tag.
        
    Out:
        url_list: list of the urls.
    """
    
    attr: dict = {attr_key : attr_value}
    url_list = []
    for page in tqdm(page_list):
        tag_list = page.find_all(tag, attr)
    
        if tag == 'a':
            for article in tag_list:
                url_list.append(article['href'])
        else :
            for article in tag_list:
                hyperlink = article.find('a') # <a> tags contain the urls
                url_list.append(hyperlink['href'])
    # Delete potential duplicates
    url_list = list(set(url_list))
    
    # Delete specific urls
    # Note that these conditions could be improven so the algorithm can delete
    # more or more specific unwanted url
    url_list_clean: list = []
    for i in range(len(url_list)):
        if ('http' not in url_list[i]):
            if (url_list[i] != '#'):
                url_list_clean.append(source_url + url_list[i])
        else :
            if source_url in url_list[i]:
                url_list_clean.append(url_list[i])
    return url_list_clean

In [40]:
#get_art_url(pages, "div",{"class":"gsc-webResult gsc-result"})     #20minutes et Journaldunet
#get_art_url(pages, "article", {"class":"item"})     #LeMondeInformatique
#get_art_url(pages, "div", {"class":"blocResultRech"})     #Usine-digitale
#test = get_art_url(page_list = pages, source_url = "https://www.usinenouvelle.com/", tag = "section", attr = {"class":"blocType1"})      #UsineNouvelle
#len(get_art_url(pages, "section", {"class":"teaser teaser--inline-picture"})) #LeMonde
#test = get_art_url(page_list = pages, source_url = "https://blockchainfrance.net/", tag = "header", attr = {"class":"entry-header th-stack--xs th-links-inherit"})
test = get_art_url(page_list = pages, source_url = "https://grh-multi.net/", 
                   tag = "header", attr_key = "class", attr_value = "entry-header")
print(test)
print(len(test))

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]

['https://grh-multi.net/fr/2017/03/master-top-10-formations-ressources-humaines-de-smbg-eduniversal/', 'https://grh-multi.net/fr/2019/04/table-ronde-1-lia-au-service-de-la-lutte-contre-les-discrimination-en-entreprise/', 'https://grh-multi.net/fr/2020/02/decouvrez-la-team-innovrh-2020/', 'https://grh-multi.net/fr/2019/04/savethedate-colloquerh/', 'https://grh-multi.net/fr/2019/05/table-ronde-2-presentation-de-nos-intervenants/', 'https://grh-multi.net/fr/2019/05/table-ronde-1-presentation-de-nos-intervenants/', 'https://grh-multi.net/fr/2019/03/stay-tuned-2/', 'https://grh-multi.net/fr/2019/03/presentation-promotion-2018-2019/', 'https://grh-multi.net/fr/2019/03/presentation-de-team-learning-expedition/', 'https://grh-multi.net/fr/2018/06/interview-etienne-audouin/']
10


Liste des mois en français (abrégé ou pas) utilisée dans la fonction scraping_research dans le cas où la date est au format "fr" (nom arbitraire donné à une date contenant les noms de mois en français, ex: 18 janvier 2019) afin de reconnaître une chaîne de caractères en tant que date au format décrit ci-avant. 


In [46]:
month_list: list = ['janv.','janvier', 
                     'févr.','février',
                     'mars',
                     'avr.','avril',
                     'mai',
                     'juin',
                     'juil.','juillet',
                      'août',
                     'sept.','septembre',
                     'oct.','octobre',
                     'nov.','novembre',
                     'déc.','décembre']

Fonction prenant en entrée une date au format "fr" et la transformant au format datetime.

In [47]:
def get_datetime(str_date: str)-> datetime:
    """Documentation
    
    Parameters:
        str_date: date written for human comprehension ("6 février 2020")
    
    Out:
        date_format_datetime: datetime corresponding to the date
    """
    trans_month = {'01':['janv.','janvier'], 
             '02':['févr.','février'],
             '03':['mars'],
             '04':['avr.','avril'],
             '05':['mai'],
             '06':['juin'],
             '07':['juil.','juillet'],
             '08':['août'],
             '09':['sept.','septembre'],
             '10':['oct.','octobre'],
             '11':['nov.','novembre'],
             '12':['déc.','décembre']}
    date_tab = str_date.split(" ")
    day = date_tab[0]
    month = date_tab[1]
    for m in trans_month:
        if month.lower() in trans_month[m]:
            month = m
    year = date_tab[2]
    date_format_datetime = date(int(year), int(month), int(day))
    return date_format_datetime


La fonction scraping_research a pour but d'utiliser la page de recherche de sites webs afin de sélectionner sur celle-ci les articles ayant une date de publication supérieure ou égale à 2020. Cela évite de scraper les pages web jugées obsolètes et ainsi d'économiser des ressources. Pour cela on récupère la liste des url des articles de la page de recherche concernée grâce à la fonction get_art_url et aux différents tags et balises renseignés en entrée de la fonction. Ensuite nous extrayons la date (format renseigné en paramètre) du contenu récupéré et la convertissons au format datetime. Tout ceci pour pouvoir faire des opérations sur les dates et ainsi récupérer seulement les plus récentes.
Il est évident que cette fonction est spécifique à un certain panel de sites car la structure des pages de recherche varie grandement d'un site web à l'autre, il est donc difficile de récupérer les dates correctement dans tous les cas. Dans les différents sites que l'on traite, on a pris la partie de récupérer  les articles qui ne présentent aucune date.
Au début du codage de la fonction nous avons pris le parti d'utiliser la fonction get_art_url afin de récupérer tous les liens des articles de la page de recherche. Cependant cette fonction a été mise à jour par la suite et ne récupérait plus certains articles jugés non pertinents (ex: les articles provenant d'un autre site web, les doublons). Cela nous a posé problème, en effet nous utilisons un compteur qui parcourt la liste url_list et celle-ci n'a donc parfois pas la même longueur que le nombre d'itérations de la balise et du tag renseignés.


In [48]:
def scraping_research(page_list: list, source_url: str, 
                      html_tag_txt: str,html_attribute_txt_key: str,
                      html_attribute_txt_value: str,
                      html_tag_articles: str, 
                      html_attribute_articles_key: str, 
                      html_attribute_articles_value: str,
                      date_format_code: str) -> list:
    """Documentation
    
    This function returns all the articles from 2020 and 2021 for a 
    website by scraping the research pages of this website. 
    It uses various tags and attributes given in the arguments 
    to find the date in the html code.
    We do that by browsing the differents pages and the different 
    articles corresponding to our research defined by our keywords list.
    
    Parameters:
    
        page_list: list of pages' source code
        source_url: base url of the website
        html_tag_txt: tag of the closest identifiable tag to the date
        html_attribute_txt: attribute associated to html_tag_txt
        html_tag_articles: tag of the closest identifiable tag to each article
        html_attribute_articles: attribute associated to html_tag_articles
        date_format_code: format of the date using the format specifiers of the datetime library
            or "fr" if the month included in the date is in french
    
    Out:
        optimised_url_list: a list of the recent articles' URL  
            (>= 2020 or without retrievable date) obtained by the research
    """
    
    html_attribute_txt: dict = {html_attribute_txt_key: html_attribute_txt_value}
    html_attribute_articles: dict = {html_attribute_articles_key: html_attribute_articles_value}
    #list of all the urls returned by the research on the pages of the website
    url_list = get_art_url(page_list, source_url, html_tag_articles, 
                           html_attribute_articles_key, html_attribute_articles_value)
    optimised_url_list = list()
    #this counter stores the number of articles already treated 
    #(added to the optimised_url_list or not) by the function
    cpt_all_articles = 0
    for page in page_list:   
        html_list = []
        #list of all the content included in each html_tag_articles html_attribute_articles
        html_list_articles = page.find_all(html_tag_articles, html_attribute_articles) 
        
        #For each html_list_articles[cpt] retrieves the part 
        #which contains the date and adds it to html_list.
        for cpt in range(0,len(html_list_articles)):
            iteration_date_article = html_list_articles[cpt].find(html_tag_txt, 
                                                                  html_attribute_txt)
            html_list.append(iteration_date_article)
            
        # for the websites "journaldunet" and "20minutes" 
        #(their research page is optimised by Google)
        # we need to remove the last term of html_list because the tag 
        #doesn't refer to an article
        if  html_tag_txt == "div" and \
        html_attribute_txt == {"class":"gs-bidi-start-align gs-snippet"}:
            
            html_list = html_list[0:len(html_tag_txt)-1]

        for article in html_list:
            #If we don't find a html_tag_txt and html_attribute_txt 
            #that corresponds to a subsection with a date 
            #we add the url to the optimised list and we go on to the next iteration
            if article is None:
                optimised_url_list.append(url_list[cpt_all_articles])
                cpt_all_articles += 1
                continue
            date_article = article.get_text().replace('\n','').replace('\xa0','')
            
            # In case the date is the only element in date_article 
            #we can directly convert it to the datetime format
            try :
                date_article = datetime.strptime(date_article, date_format_code).date()
            # Otherwise we have to extract the date from the content, 
            #using various techniques depending on the website
            except :
                #Here we use a different date_format_code that 
                #the one specified in the input (only case we do that)
                if 'Mis à jour le' in date_article and \
                html_attribute_txt == {"class":"gs-bidi-start-align gs-snippet"}:
                    
                    date_article = date_article.split('Mis à jour le ')[1].split()[0]
                    date_article = datetime.strptime(date_article,'%d/%m/%y').date()
                    
                else:
                    if 'Publié le' in date_article:
                        date_article = date_article.split('Publié le ')[0]
                    date_article = date_article.split()
                    # We test if the beginning of date_article is a date 
                    #in the 'fr' format and converts it to datetime format 
                    if len(date_article[0]) in [1,2] and \
                    date_article[1] in month_list and len(date_article[2]) == 4:
                        
                        date_article = get_datetime(date_article[0] + ' ' + date_article[1] + ' ' + date_article[2])
                    # This else corresponds to an article without date. 
                    #We assign to date_articles an arbitrary date >= 2020  
                    else:
                        date_article = date(2020, 2, 27)
                        
            # Retrieves dates that are >= 2020            
            if date_article > date(2019, 12, 31):
                optimised_url_list.append(url_list[cpt_all_articles])
            cpt_all_articles += 1
            
    return optimised_url_list

print(scraping_research(pages, 'https://grh-multi.net/', 'time' , 
                        "class", "entry-date published",'header', 
                        "class", "entry-header", "fr" ))



100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 157.26it/s]

['https://grh-multi.net/fr/2019/04/table-ronde-1-lia-au-service-de-la-lutte-contre-les-discrimination-en-entreprise/']


## Conclusion 

Les résultats que nous montrons dans ce document n'aboutissent pas à une généralisation du processus. Chaque site propose une architecture et un emplacement de l'information trop différent les uns des autres pour être généraliser. Néanmoins, une construction spécifique par site est possible afin de faire cette sélection par date en pré-scrapping. 
La sélection par date est, en effet, difficilement implémentable dans une solution comme celle décrite dans ce document de recherche de nouveaux articles directement sur le site web visé. 
Une autre solution a été abordée durant le projet afin de généraliser cette recherche d'articles. Une solution moteur de recherche permet de faire une sélection par date et permettant ainsi de répondre à la problématique donnée [voir Poc g2_poc_crawler_API_google_quotidien.ipynb].
